In [6]:
import geopandas as gpd
import folium
import pandas as pd
from folium import Element

# ==================== STATE MAP ====================

# Load state data
new_england = gpd.read_file('shapefiles/NEWENGLAND_POLY.shp')
parameters_df = pd.read_csv('New_England_Datasets/NEW ENGLAND\'s data (states) - NEW ENGLAND.csv')

# Clean and merge state data
new_england = new_england.dissolve(by='NAME', aggfunc='first').reset_index()

parameters_df['Population'] = parameters_df['Population'].str.replace(',', '').astype(float)
parameters_df['Income'] = parameters_df['Income (per capita)'].str.replace('$', '').str.replace(',', '').astype(float)
parameters_df['Education'] = parameters_df['Education \r\n(high school or higher attendance rate)'].str.replace('%', '').astype(float)
parameters_df['Health'] = pd.to_numeric(parameters_df['Health (life expectancy)'], errors='coerce')
parameters_df['Crime Rate'] = pd.to_numeric(parameters_df['Crime rate (per population)'], errors='coerce')

new_england['Population'] = parameters_df['Population'].values
new_england['Income'] = parameters_df['Income'].values
new_england['Education'] = parameters_df['Education'].values
new_england['Health'] = parameters_df['Health'].values
new_england['Crime Rate'] = parameters_df['Crime Rate'].values

new_england = new_england.to_crs(epsg=4326)

# Create state map
m = folium.Map(location=[44, -71], zoom_start=6)

# Add Income choropleth
folium.Choropleth(
    geo_data=new_england,
    name='Income',
    data=new_england,
    columns=['NAME', 'Income'],
    key_on='feature.properties.NAME',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Income per Capita ($)',
    overlay=True,
    show=True
).add_to(m)

# Add Education choropleth
folium.Choropleth(
    geo_data=new_england,
    name='Education',
    data=new_england,
    columns=['NAME', 'Education'],
    key_on='feature.properties.NAME',
    fill_color='Blues',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Education (%)',
    overlay=True,
    show=False
).add_to(m)

# Add Healthcare choropleth
folium.Choropleth(
    geo_data=new_england,
    name='Healthcare',
    data=new_england,
    columns=['NAME', 'Health'],
    key_on='feature.properties.NAME',
    fill_color='Purples',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Life Expectancy',
    overlay=True,
    show=False
).add_to(m)

# Add Crime Rate choropleth
folium.Choropleth(
    geo_data=new_england,
    name='Crime Rate',
    data=new_england,
    columns=['NAME', 'Crime Rate'],
    key_on='feature.properties.NAME',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Crime Rate',
    overlay=True,
    show=False
).add_to(m)

# Add Population choropleth
folium.Choropleth(
    geo_data=new_england,
    name='Population',
    data=new_england,
    columns=['NAME', 'Population'],
    key_on='feature.properties.NAME',
    fill_color='Oranges',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Population',
    overlay=True,
    show=False
).add_to(m)

# Map state names to county map files
state_to_file = {
    'CONNECTICUT': 'ct_counties_map.html',
    'MAINE': 'me_counties_map.html',
    'MASSACHUSETTS': 'mass_counties_map.html',
    'NEW HAMPSHIRE': 'nh_counties_map.html',
    'RHODE ISLAND': 'ri_counties_map.html',
    'VERMONT': 'vt_counties_map.html'
}

# Add clickable popups to each state
for idx, row in new_england.iterrows():
    state_name = row['NAME']
    county_file = state_to_file.get(state_name, '#')
    
    popup_html = f"""
    <div style="font-family: Arial; width: 220px; text-align: center; padding: 10px;">
        <h3 style="margin: 5px 0; color: #2c3e50;">{state_name}</h3>
        <hr style="margin: 10px 0;">
        <p style="margin: 5px;"><b>Population:</b> {row['Population']:,.0f}</p>
        <p style="margin: 5px;"><b>Income:</b> ${row['Income']:,.0f}</p>
        <p style="margin: 5px;"><b>Education:</b> {row['Education']:.1f}%</p>
        <p style="margin: 5px;"><b>Life Expectancy:</b> {row['Health']:.1f} years</p>
        <hr style="margin: 10px 0;">
        <a href="{county_file}" 
           target="_blank"
           style="
               background-color: #4CAF50;
               color: white;
               padding: 12px 24px;
               text-decoration: none;
               border-radius: 5px;
               display: inline-block;
               font-weight: bold;
               margin-top: 5px;
           ">
           View Counties →
        </a>
    </div>
    """
    
    folium.GeoJson(
        row['geometry'],
        popup=folium.Popup(popup_html, max_width=260),
        style_function=lambda x: {
            'fillOpacity': 0,
            'color': 'transparent',
            'weight': 0
        }
    ).add_to(m)

# Add hover tooltips
tooltip_layer = folium.FeatureGroup(name='State Info', overlay=True, control=True, show=True)

folium.GeoJson(
    new_england,
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME', 'Population', 'Income', 'Education', 'Health', 'Crime Rate'],
        aliases=['State:', 'Population:', 'Income/capita:', 'Education %:', 'Life Expectancy:', 'Crime Rate:'],
        style='background-color: white; color: black; font-size: 12px; padding: 10px;',
        sticky=False
    ),
    style_function=lambda x: {
        'fillOpacity': 0,
        'color': 'black',
        'weight': 0.5,
        'zIndex': 1000
    },
    highlight_function=lambda x: {
        'weight': 2,
        'color': 'yellow'
    }
).add_to(tooltip_layer)

tooltip_layer.add_to(m)

folium.LayerControl().add_to(m)

# Save state map
m.save('maps/new_england_map.html')
print("✅ State map created: maps/new_england_map.html")

# ==================== COUNTY MAPS ====================

# Define states with their FIPS codes and file paths
states_info = {
    'Connecticut': {
        'fips': '09', 
        'abbr': 'ct', 
        'shapefile': 'ct_county_shapefiles/ct_counties.shp', 
        'data': "New_England_Datasets/NEW ENGLAND's data (states) - CT.csv"
    },
    'Maine': {
        'fips': '23', 
        'abbr': 'me', 
        'shapefile': 'me_county_shapefiles/me_counties.shp', 
        'data': "New_England_Datasets/NEW ENGLAND's data (states) - ME.csv"
    },
    'Massachusetts': {
        'fips': '25', 
        'abbr': 'mass', 
        'shapefile': 'ma_county_shapefiles/ma_counties.shp',  # FIXED PATH
        'data': "New_England_Datasets/NEW ENGLAND's data (states) - MA.csv"
    },
    'New Hampshire': {
        'fips': '33', 
        'abbr': 'nh', 
        'shapefile': 'nh_county_shapefiles/nh_counties.shp', 
        'data': "New_England_Datasets/NEW ENGLAND's data (states) - NH.csv"
    },
    'Rhode Island': {
        'fips': '44', 
        'abbr': 'ri', 
        'shapefile': 'ri_county_shapefiles/ri_counties.shp', 
        'data': "New_England_Datasets/NEW ENGLAND's data (states) - RI.csv"
    },
    'Vermont': {
        'fips': '50', 
        'abbr': 'vt', 
        'shapefile': 'vt_county_shapefiles/vt_counties.shp', 
        'data': "New_England_Datasets/NEW ENGLAND's data (states) - VT.csv"
    }
}

back_button_html = """
<div style="position: fixed; top: 10px; left: 60px; z-index: 1000;">
    <a href="new_england_map.html" 
       style="
           background-color: #2196F3;
           color: white;
           padding: 10px 20px;
           text-decoration: none;
           border-radius: 5px;
           font-family: Arial;
           font-weight: bold;
           box-shadow: 0 2px 5px rgba(0,0,0,0.3);
       ">
       ← Back to New England
    </a>
</div>
"""

# Create county map for each state
for state_name, info in states_info.items():
    try:
        print(f"Creating map for {state_name}...")

        if state_name=="Massachusetts":
            counties = gpd.read_file('counties_shapefiles/COUNTIES_POLYM.shp')
            if 'COUNTY' in counties.columns:
                counties = counties.rename(columns={'COUNTY': 'NAME'})
            elif 'County' in counties.columns:
                counties = counties.rename(columns={'County': 'NAME'})
        
        
        else:
            counties = gpd.read_file(info['shapefile'])
        
        # Load county data
        county_data = pd.read_csv(info['data'])
        
        # Clean data - NO need to uppercase, it's already 'COUNTY'
        county_data['Population'] = county_data['Population'].str.replace(',', '').astype(float)
        county_data['Income'] = county_data['Income (per capita)'].str.replace('$', '').str.replace(',', '').astype(float)
        county_data['Education'] = county_data['Education \n(high school or higher attendance rate)'].str.replace('%', '').astype(float)
        county_data['Health'] = pd.to_numeric(county_data['Health (life expectancy)'], errors='coerce')
        county_data['Crime Rate'] = pd.to_numeric(county_data['Crime rate (per population)'], errors='coerce')
        
        # Merge data - USE 'COUNTY' NOT 'County'
        counties = counties.merge(
            county_data[['COUNTY', 'Population', 'Income', 'Education', 'Health', 'Crime Rate']],
            left_on='NAME',
            right_on='COUNTY',
            how='left'
        )
        
        counties = counties.to_crs(epsg=4326)
        
        # Create county map
        county_map = folium.Map(location=[44, -71], zoom_start=7)
        
        # Add Income choropleth
        folium.Choropleth(
            geo_data=counties,
            name='Income',
            data=counties,
            columns=['NAME', 'Income'],
            key_on='feature.properties.NAME',
            fill_color='YlGn',
            fill_opacity=0.7,
            line_opacity=0.5,
            legend_name='Income per Capita ($)',
            overlay=True,
            show=True
        ).add_to(county_map)
        
        # Add Education choropleth
        folium.Choropleth(
            geo_data=counties,
            name='Education',
            data=counties,
            columns=['NAME', 'Education'],
            key_on='feature.properties.NAME',
            fill_color='Blues',
            fill_opacity=0.7,
            line_opacity=0.5,
            legend_name='Education (%)',
            overlay=True,
            show=False
        ).add_to(county_map)
        
        # Add Healthcare choropleth
        folium.Choropleth(
            geo_data=counties,
            name='Healthcare',
            data=counties,
            columns=['NAME', 'Health'],
            key_on='feature.properties.NAME',
            fill_color='Purples',
            fill_opacity=0.7,
            line_opacity=0.5,
            legend_name='Life Expectancy',
            overlay=True,
            show=False
        ).add_to(county_map)
        
        # Add Crime Rate choropleth
        folium.Choropleth(
            geo_data=counties,
            name='Crime Rate',
            data=counties,
            columns=['NAME', 'Crime Rate'],
            key_on='feature.properties.NAME',
            fill_color='Reds',
            fill_opacity=0.7,
            line_opacity=0.5,
            legend_name='Crime Rate',
            overlay=True,
            show=False
        ).add_to(county_map)
        
        # Add Population choropleth
        folium.Choropleth(
            geo_data=counties,
            name='Population',
            data=counties,
            columns=['NAME', 'Population'],
            key_on='feature.properties.NAME',
            fill_color='Oranges',
            fill_opacity=0.7,
            line_opacity=0.5,
            legend_name='Population',
            overlay=True,
            show=False
        ).add_to(county_map)
        
        # Add tooltips
        tooltip_layer = folium.FeatureGroup(name='County Info', overlay=True, control=True, show=True)
        
        folium.GeoJson(
            counties,
            tooltip=folium.GeoJsonTooltip(
                fields=['NAME', 'Population', 'Income', 'Education', 'Health', 'Crime Rate'],
                aliases=['County:', 'Population:', 'Income:', 'Education %:', 'Life Expectancy:', 'Crime Rate:'],
                style='background-color: white; color: black; font-size: 12px; padding: 10px;'
            ),
            style_function=lambda x: {
                'fillOpacity': 0,
                'color': 'black',
                'weight': 0.5
            }
        ).add_to(tooltip_layer)
        
        tooltip_layer.add_to(county_map)
        
        # Add layer control
        folium.LayerControl().add_to(county_map)
        
        # Add back button
        county_map.get_root().html.add_child(Element(back_button_html))
        
        # Save county map
        output_file = f"maps/{info['abbr']}_counties_map.html"
        county_map.save(output_file)
        print(f"✅ {state_name} county map created: {output_file}")
        
    except Exception as e:
        print(f"❌ Error creating map for {state_name}: {e}")

print("\n🎉 All maps created successfully!")
print("📍 Open maps/new_england_map.html to start exploring!")

✅ State map created: maps/new_england_map.html
Creating map for Connecticut...
✅ Connecticut county map created: maps/ct_counties_map.html
Creating map for Maine...
✅ Maine county map created: maps/me_counties_map.html
Creating map for Massachusetts...
✅ Massachusetts county map created: maps/mass_counties_map.html
Creating map for New Hampshire...
✅ New Hampshire county map created: maps/nh_counties_map.html
Creating map for Rhode Island...
✅ Rhode Island county map created: maps/ri_counties_map.html
Creating map for Vermont...
✅ Vermont county map created: maps/vt_counties_map.html

🎉 All maps created successfully!
📍 Open maps/new_england_map.html to start exploring!
